In [74]:
import requests
import random
import string
import time
import json

### helpers

In [28]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

In [34]:
with open('/usr/share/dict/words', 'r') as f:
    words_list = f.read().strip().splitlines()

### signup

In [29]:
user_passwords = {}
batch_id = int(time.time())
for i in range(1, 21):
    name = 'user {}.{}'.format(batch_id, str(i).zfill(2))
    username = '{}@email.com'.format(name.replace(' ','.'))
    password = 'password'
    user_cred = {
        'display_name': name, 
        'username': username,
        'password': password
    }
    
    r = requests.post('http://localhost:8080/auth/signup', json=user_cred)
    assert r.status_code in range(200,300), 'failed {}'.format(user_cred['username'])
    user_passwords[username] = password

### auth

In [30]:
user_tokens = {}

for username, password in user_passwords.items():
    user_cred = {'username': username, 'password': password}
    r = requests.post('http://localhost:8080/auth/login', json=user_cred)
    assert r.status_code in range(200,300), 'failed {}'.format(username)
    username = user_cred['username'].lower()
    user_tokens[username] = r.json()['token']
    
print('success')

success


### get users

In [31]:
def make_auth_headers(token):
    return {'authorization': 'bearer {}'.format(token)}

In [80]:
auth_headers = make_auth_headers(random.choice(list(user_tokens.values())))
r = requests.get('http://localhost:8080/user/',headers=auth_headers)
user_data = {u['username']: u for u in r.json()['users']}
user_data = {k: v for k,v in user_data.items() if k in user_tokens}

### create posts

In [33]:
tag_names = [
    'apps',
    'aws',
    'containers',
    'docker',
    'healthtech',
    'hippa',
    'jill',
    'tutorial'
]

In [79]:
for i in range(10):
    current_users = [username for username in k_unique(list(user_data.keys()), random.randint(1,7))]
    author_username = current_users[0]
    auth_headers = make_auth_headers(user_tokens[author_username])
    collaborators = [user_data[username]['user_id'] for username in current_users]
    
    title = ' '.join(random.choices(words_list, k=random.randint(3,10)))
    body = {}
    for i in range(random.randint(2,30)):
        body[str(i)] = ' '.join(random.choices(words_list, k=random.randint(3,10))).title() + '. '
    body = json.dumps(body)
    
    explicit_tags = k_unique(tag_names, k=3)
    
    post = {
        'title': title,
        'body': body,
        'explicit_tags': explicit_tags,
        'collaborators': collaborators
    }
    
    r = requests.post('http://localhost:8080/post/', json=post, headers = auth_headers)
    assert r.status_code in range(200,300), 'bad status'